# Прямая задача кинематики

In [ ]:
from matplotlib import pyplot as plt
from matplotlib import animation
import numpy as np
from IPython.display import HTML
%matplotlib notebook

In [ ]:
from kinematics import Vector, Quaternion, Transform
import graphics

## Решение прямой задачи кинематики для манипулятора SCARA

Манипулятор кинематической схемы SCARA обладает четыремя степенями подвижности.

![внеший вид SCARA](fig/scara_model.png)

Можно выделить высоту колонны, длины первого и второго звеньев.

Обобщенные координаты будут в радианах и метрах:

Обобщенная координата | Обозначение | Размерность
---|---|---
Вращение вокруг колонны | $q_0$ | радиан
Вращение в локте | $q_1$ | радиан
Вращение инструмента | $q_2$ | радиан
Перемещение инструмента | $q_3$ | метр

In [ ]:
def scara_chain(q, l):
    base = Transform.identity()
    column = base + Transform(
        Vector(0, 0, l[0]),
        Quaternion.from_angle_axis(q[0], Vector(0, 0, 1))
    )
    elbow = column + Transform(
        Vector(l[1], 0, 0),
        Quaternion.from_angle_axis(q[1], Vector(0, 0, 1))
    )
    tool = elbow + Transform(
        Vector(l[2], 0, 0),
        Quaternion.from_angle_axis(q[2], Vector(0, 0, 1))
    )
    flange = tool + Transform(
        Vector(0, 0, -q[3]),
        Quaternion.identity()
    )
    return [
        base,
        column,
        elbow,
        tool,
        flange
    ]

Зададим закон изменения обобщенных координат:

In [ ]:
def scara_q(t, total):
    omega = t / total * np.pi * 2
    return [
        np.pi / 4 * np.sin(omega),
        np.pi / 2,
        omega,
        3 + 3 * np.sin(omega)
    ]

Укажем длины звеньев:

In [ ]:
scara_l = [8, 4, 3]

In [ ]:
scara_fig = plt.figure()
ax = scara_fig.add_subplot(projection="3d")
ax.set_xlim(-6, 6); ax.set_ylim(-6, 6); ax.set_zlim(0, 12)
lines, = ax.plot([], [], [], color="#000000")
graphics.axis(ax, Transform.identity(), 2)
r, g, b = graphics.axis(ax, Transform.identity(), 1)
def animate(i):
    omega = i / 100 * np.pi * 2
    chain = scara_chain(scara_q(i, 100), scara_l)
    (x, y, z) = graphics.chain_to_points(chain)
    lines.set_data_3d(x, y, z)
    global r, g, b
    r.remove(); g.remove(); b.remove()
    r, g, b = graphics.axis(ax, chain[-1], 0.5)

    
animate(0)
fps = 25
scara_ani = animation.FuncAnimation(
    scara_fig,
    animate,
    frames=100,
    interval=1000.0/fps
)

In [ ]:
HTML(scara_ani.to_jshtml())

## Решение прямой задачи кинематики для манипулятора PUMA

Манипулятор кинематической схемы PUMA обладает шестью степенями подвижности.

![внешний виж PUMA](fig/puma_model.png)

Его можно условно разделить на сегменты, соединияюще между собой:
- основание
- плечо
- локоть
- кисть
- фланец

![кинематическая схема PUMA](fig/puma.png)

Длина этих сегментов определена в таблице:

Пара | длина
:----|:-----
основание - плечо | $l_0$
плечо - локоть | $l_1$
локоть - кисть | $l_2$
кисть - фланец | $l_3$

In [ ]:
def puma_chain(q, l):
    base = Transform.identity()
    shoulder = base + Transform(
        Vector(0, 0, l[0]),
        Quaternion.from_angle_axis(q[0], Vector(0, 0, 1)) * 
        Quaternion.from_angle_axis(q[1], Vector(0, 1, 0))
    )
    elbow = shoulder + Transform(
        Vector(0, 0, l[1]),
        Quaternion.from_angle_axis(q[2], Vector(0, 1, 0))
    )
    wrist = elbow + Transform(
        Vector(0, 0, l[2]),
        Quaternion.from_angle_axis(q[3], Vector(0, 0, 1)) *
        Quaternion.from_angle_axis(q[4], Vector(0, 1, 0))
    )
    flange = wrist + Transform(
        Vector(0, 0, l[3]),
        Quaternion.from_angle_axis(q[5], Vector(0, 0, 1))
    )
    return [base, shoulder, elbow, wrist, flange]

Зададим изменение обобщенных координат:

In [ ]:
def puma_q(t, total):
    omega = t / total * np.pi * 2
    return [
        np.pi / 4 * np.sin(omega),
        np.pi / 8,
        np.pi / 2,
        omega,
        np.pi / 2,
        0
    ]

Зададим длины звеньев:

In [ ]:
puma_l = [1, 2, 1, 0.5]

In [ ]:
puma_fig = plt.figure()
ax = puma_fig.add_subplot(projection="3d")
ax.set_xlim([-2, 2]); ax.set_ylim([-2, 2]); ax.set_zlim([0, 4])
lines, = ax.plot([], [], [], color="#000000")
graphics.axis(ax, Transform.identity(), 2)
r, g, b = graphics.axis(ax, Transform.identity(), 1)
def animate(i):
    omega = i / 100 * np.pi * 2
    chain = puma_chain(puma_q(i, 100), puma_l)
    (x, y, z) = graphics.chain_to_points(chain)
    lines.set_data_3d(x, y, z)
    global r, g, b
    r.remove(); g.remove(); b.remove()
    r, g, b = graphics.axis(ax, chain[-1], 0.5)

    
animate(0)
fps = 25
puma_ani = animation.FuncAnimation(
    puma_fig,
    animate,
    frames=100,
    interval=1000.0/fps
)

In [ ]:
HTML(puma_ani.to_jshtml())